In [21]:
from pathlib import Path

import numpy as np
from numba import njit, prange

import pandas as pd

In [22]:
df = pd.read_csv(Path('..', '..', 'data', 'iris_csv.csv'))

for c in df.columns[0:4]:
    df[c] = (df[c]-df[c].mean())/df[c].std()

df['synth1'] = df['petallength']*df['petalwidth']
df['synth2'] = df['sepallength']*df['petallength']
df['synth3'] = df['sepallength']*df['petalwidth']

for name in df['class'].unique():
    df[f'label-{name}'] = df['class'].map(lambda x: 1 if x == name else 0)

In [23]:
np.random.seed(0)

setosa_idxs = np.arange(0, 50)
versicolor_idxs = np.arange(50, 100)
virginica_idxs = np.arange(100, 150)

p = np.random.permutation(np.arange(50))

setosa_train_idxs = setosa_idxs[p[0:10]]
setosa_test_idxs = setosa_idxs[p[10:]]

versicolor_train_idxs = versicolor_idxs[p[0:10]]
versicolor_test_idxs = versicolor_idxs[p[10:]]

virginica_train_idxs = virginica_idxs[p[0:10]]
virginica_test_idxs = virginica_idxs[p[10:]]

feature_columns = ['sepallength', 'sepalwidth', 'petallength', 'petalwidth']
label_columns = ['label-Iris-setosa', 'label-Iris-versicolor', 'label-Iris-virginica']

xTrain = np.vstack([
    df.iloc[setosa_train_idxs][feature_columns],
    df.iloc[versicolor_train_idxs][feature_columns],
    df.iloc[virginica_train_idxs][feature_columns]
])

yTrain = np.vstack([
    df.iloc[setosa_train_idxs][label_columns],
    df.iloc[versicolor_train_idxs][label_columns],
    df.iloc[virginica_train_idxs][label_columns]
])

xTest = np.vstack([
    df.iloc[setosa_test_idxs][feature_columns],
    df.iloc[versicolor_test_idxs][feature_columns],
    df.iloc[virginica_test_idxs][feature_columns]
])

yTest = np.vstack([
    df.iloc[setosa_test_idxs][label_columns],
    df.iloc[versicolor_test_idxs][label_columns],
    df.iloc[virginica_test_idxs][label_columns]
])

In [24]:
@njit(fastmath=True)
def F(x: np.ndarray) -> np.ndarray:
    # return x
    # return np.maximum(np.zeros(x.shape), x)
    # return np.clip(x,-1, 1)
    return np.tanh(x)


@njit(fastmath=True)
def dF(x: np.ndarray) -> np.ndarray:
    # return np.ones(x.shape)
    # return 1 * (x > 0)
    # return np.array([0 if xi <= -1 or xi >= 1 else 1 for xi in x])
    return 1-np.square(np.tanh(x))


@njit(fastmath=True)
def sigmoid(x: np.ndarray) -> np.ndarray:
    return 1/(1+np.exp(-x))


@njit(fastmath=True)
def dSigmoid(x: np.ndarray) -> np.ndarray:
    y = 1/(1+np.exp(-x))
    return y*(1-y)


@njit(fastmath=True)
def softmax(x: np.ndarray) -> np.ndarray:
    y = np.exp(x)
    return y/np.sum(y)

In [25]:
@njit(fastmath=True)
def grads(xBatch: np.ndarray, w: np.ndarray, Bh:np.ndarray, b:np.ndarray) -> tuple[np.ndarray]:
    dw = np.zeros(w.shape)
    dBh = np.zeros(Bh.shape)
    db = np.zeros(b.shape)
    
    for i in prange(xBatch.shape[0]):
        u = xBatch[i] @ w + Bh
        y = F(u) @ w.T + b

        dLdy = 2/(w.shape[0]* w.shape[1]) * (y-xBatch[i])
        
        dw += np.outer(xBatch[i], dLdy @ w) * dF(u)
        
        dBh += (dLdy @ w) * dF(u)
        db += dLdy
    
    return (dw, dBh, db)


class RestrictedBoltzmannMachine:
    def __init__(self, nIn: int, nHidden: int) -> None:
        self.nIn = nIn
        self.nHidden = nHidden
        
        self.w: np.ndarray = np.random.uniform(-1, 1, (nIn, nHidden))
        
        self.Bh: np.ndarray = np.zeros(nHidden)
        self.b: np.ndarray = np.zeros(nIn)


    def predict(self, x:np.ndarray) -> np.ndarray:
        # return (x @ self.w + self.Bh) @ self.w.T + self.b
        return F(x @ self.w + self.Bh) @ self.w.T + self.b


    def train(self, xTrain: np.ndarray, lr, batch_size, max_iter) -> None:
        n = xTrain.shape[0]

        for k in range(max_iter):
            idxs = np.random.choice(a=np.arange(n), size=batch_size, replace=False)
            
            dw, dBh, db = grads(xTrain[idxs], self.w, self.Bh, self.b)
            
            self.w -= lr*dw
            self.Bh -= lr*dBh
            self.b -= lr*db
        
    
    def loss(self, x: np.ndarray) -> float:
        xPred = np.array([self.predict(xi) for xi in x])
        d = 1/self.nIn * np.linalg.norm(x-xPred, axis=1)
        return 1/x.shape[0] * np.sum(d)

In [26]:
nIn = 4
nHidden = 2

lr = 1e-2
batch_size = 30
max_iter = 5000

model = RestrictedBoltzmannMachine(nIn, nHidden)

print('untrained loss: {0:.6f}'.format(model.loss(xTest)))

model.train(xTrain, lr, batch_size, max_iter)

print('trained loss: {0:.6f}'.format(model.loss(xTest)))

untrained loss: 0.318504
trained loss: 0.374042


In [27]:
for x in xTest:
    xPred = model.predict(x)
    loss = 1/model.nHidden * np.linalg.norm(x-xPred).round(3)
    print(round(loss,2), x.round(2), xPred.round(2))

1.26 [-0.41  2.64 -1.34 -1.31] [ 0.55  1.19  0.09 -0.19]
1.1 [-1.02  0.34 -1.45 -1.31] [-0.1   0.29 -0.5   0.45]
0.97 [-1.02  0.8  -1.22 -1.05] [-0.    0.59 -0.51  0.43]
1.13 [-1.14  0.11 -1.28 -1.44] [-0.13 -0.02 -0.35  0.35]
0.91 [-0.17  1.72 -1.17 -1.18] [ 0.47  1.1  -0.01 -0.09]
1.08 [-1.02  0.8  -1.28 -1.31] [ 0.05  0.62 -0.43  0.36]
1.11 [-0.05  2.18 -1.45 -1.31] [ 0.53  1.17  0.06 -0.16]
1.06 [-1.26 -0.12 -1.34 -1.18] [-0.26 -0.28 -0.42  0.44]
1.04 [-0.66  1.49 -1.28 -1.31] [ 0.31  0.96 -0.2   0.11]
1.14 [-1.38  0.34 -1.22 -1.31] [-0.13  0.16 -0.47  0.44]
1.3 [-0.17  3.1  -1.28 -1.05] [ 0.61  1.23  0.16 -0.25]
1.1 [-1.26  0.11 -1.22 -1.31] [-0.17 -0.06 -0.4   0.4 ]
1.29 [-0.78  2.41 -1.28 -1.44] [ 0.49  1.14  0.01 -0.11]
1.04 [-0.54  1.95 -1.39 -1.05] [ 0.31  1.   -0.22  0.12]
1.27 [-1.74  0.34 -1.39 -1.31] [-0.21  0.12 -0.58  0.55]
0.95 [-0.54  0.8  -1.17 -1.31] [ 0.22  0.75 -0.23  0.15]
0.89 [-1.02  1.03 -1.22 -0.78] [-0.03  0.66 -0.59  0.5 ]
1.22 [-1.74 -0.36 -1.34 -1.31] [-0